In [5]:
import pyarrow.parquet as pq
import re


parquet_file = pq.read_table('CESNET_domains.parquet')

#print number of rows
print("Number of rows: ", parquet_file.num_rows)

#save it to txt 
with open('CESNET_domains.txt', 'w') as f:
    for i in range(0,parquet_file.num_rows):
        #print(parquet_file.column(0)[i])
        f.write(str(parquet_file.column(0)[i]))
        f.write("\n")

#print 10 first rows
print("10 first rows: ", parquet_file.column(0)[0:10])


Number of rows:  529384
10 first rows:  [
  [
    "000000.pn01.edugroup.at",
    "000000.tpn01.edugroup.at",
    "00000.cz",
    "0:0:0:0:0:FFFF:B76E:5A27",
    "00000fs3085d.han2.savba.sk",
    "0000100b00e9.han2.savba.sk",
    "0000100b01bd.han2.savba.sk",
    "0000100m0030.han2.savba.sk",
    "0000100q012c.han2.savba.sk",
    "0000100q0296.han2.savba.sk"
  ]
]


In [1]:
import dask.dataframe as dd
import tldextract
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

cutoff = 50
def extract_domain(x):
    ext = tldextract.extract(x)
    return ext.registered_domain if ext.registered_domain else x

# Read the Parquet file using Dask DataFrame.
ddf = pq.read_table('CESNET_domains.parquet')

# Apply the extract_domain function to the 'DOMAIN' column.
ddf['extracted'] = ddf['DOMAIN'].map(extract_domain, meta=('extracted', 'object'))

# Filter rows where the domain is invalid or could not be extracted.
ddf = ddf[ddf['extracted'] != '']

# Since Dask operations are lazy, up to this point, no computations have been performed.
# Now, let's perform a streaming computation to get the top domains by suffix.
top_domains_by_suffix = ddf.groupby('extracted').size().nlargest(cutoff).compute()

# Get the list of top domains.
top_domains_list = top_domains_by_suffix.index.tolist()

# Now filter the original DDF to only include the top domains.
filtered_ddf = ddf[ddf['extracted'].isin(top_domains_list)]

# Perform a random sample of the filtered DDF.
sampled_ddf = filtered_ddf.sample(frac=(600000 / len(filtered_ddf)), random_state=42)

# Write the sampled data directly to a text file, one domain name per line.
# This operation is lazy and will be computed only when triggered.
fraction_to_sample = 600_000 / len(filtered_ddf)

# Perform a random sample of the filtered DDF.
# The `random_state` ensures that the sample is reproducible.
sampled_ddf = filtered_ddf.sample(frac=fraction_to_sample, random_state=42)

# This writes the sampled data directly to a text file, one domain name per line.
# The asterisk (*) in the filename will be replaced with numbers to denote different parts if needed.
# We ensure that each partition writes to its own file to avoid memory issues.
paths = sampled_ddf['extracted'].to_textfiles('domains_sampled_*.txt', index=False)


# import os
# os.system("cat domains_sampled_*.txt > combined_domains.txt")

# # After this step, you can delete the part files if you want to keep only the combined file.
# for path in paths:
#     os.remove(path)

/home/survivecz/.local/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


AttributeError: 'pyarrow.lib.ChunkedArray' object has no attribute 'map'